In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import SummaryIndex
from llama_index.core import Settings
from llama_index.core.tools import QueryEngineTool

from vllmpoint import get_vllm_embedder, get_vllm_llm

import logging
import sys

In [17]:
logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [18]:
llm = get_vllm_llm()
embed_model = get_vllm_embedder()

In [19]:
Settings.llm = llm
Settings.embed_model = embed_model

In [22]:
# Simple query by indexing
# TODO: Try multimedia inputs like mp3 and mp4
documents = SimpleDirectoryReader(
    input_files=["feedback.docx"]
).load_data()
feedback_index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"


In [23]:
query = "What is the feedback about? Give me names."
query_engine = feedback_index.as_query_engine()
answer = query_engine.query(query)

INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"


In [24]:
print(answer.get_formatted_sources())
print("Query was:", query)
print("Answer was:", answer)

> Source (Doc id: ec5fd2e7-010a-4a7e-8fb5-36707a3e5bc3): Interview Feedback Form - Qylis

Section 1: Candidate Details 

Name of Candidate   : ______ ____...
Query was: What is the feedback about? Give me names.
Answer was: The feedback is about an interview for a Data Science position. The names mentioned are Sohel and Akshay.


In [ ]:
# Summarization
summary_index = SummaryIndex.from_documents(documents)

query_engine = summary_index.as_query_engine(response_mode="tree_summarize", streaming=True)
response = query_engine.query("<summarization_query>")
response.print_response_stream()
print('')

INFO:openai._base_client:Retrying request to /chat/completions in 0.468332 seconds
Retrying request to /chat/completions in 0.468332 seconds
Retrying request to /chat/completions in 0.468332 seconds
Retrying request to /chat/completions in 0.468332 seconds


In [ ]:
documents = SimpleDirectoryReader(
    input_files=["paul_graham_essay.txt"]
).load_data()
essay_index = VectorStoreIndex.from_documents(documents)

# Define query engines and tools
tool1 = QueryEngineTool.from_defaults(
    query_engine=feedback_index.as_query_engine(),
    description="Use this query engine for feedback form.",
    # description="Use this query engine for an English essay.", Switch
)
tool2 = QueryEngineTool.from_defaults(
    query_engine=essay_index.as_query_engine(),
    description="Use this query engine for an English essay.",
    # description="Use this query engine for feedback form.", Switch
)

In [ ]:
# Get as chat. Keeps track of conversation history
chat_engine = feedback_index.as_chat_engine()
chat1 = "Who is the interview candidate?"
response1 = chat_engine.chat(chat1)
print('Response1:', response1)
chat2 = "Who is the interviewer?"
response2 = chat_engine.chat(chat2)
print('Response2:', response2)

In [ ]:
chat3 = "Give me the summary of the feedback?"
response3 = chat_engine.chat(chat3)
print(response3)

In [ ]:
# For interactive chat
chat_engine = feedback_index.as_chat_engine()
chat_engine.chat_repl()